<a href="https://colab.research.google.com/github/hyojgun/hyojeong/blob/main/kobert_ft1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#data download
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1 (from 1)
Receiving objects: 100% (14763/14763), 56.19 MiB | 16.98 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.


In [2]:
import pandas as pd

df = pd.read_csv("./nsmc/ratings.txt", delimiter='\t', quoting=3); df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [3]:
#total  200,000 rows
import pandas as pd

df = pd.read_csv("./nsmc/ratings.txt", delimiter='\t', quoting=3); df.info()
df.head(10)

# label이 0인 부정적인 문장만 필터링
negative_df = df[df['label'] == 0]

# 부정적인 문장 상위 10개 출력
negative_df.head(10)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


,id,document,label
100000,6826470,이런영화로 관객들한테 돈벌고싶소?,0
100001,6239594,작품 선구안이 없다는게 배우 김태희의 최대 약점.,0
100002,8946612,사극?? 로멘스?? 퓨젼??,0
100003,4800899,ㅋㅋㅋㅋ엿국니네가그렇지므ㅝ 이건뭐 영화도아니고,0
100004,5927640,완죤유치하고 못봐주겠네...,0
100005,3619043,"""""""이지현,님의 Body 는 환상이었다.""",0
100006,7505472,재미없어 연기 넘 오글거려 ㅋㅋㅋ,0
100007,5785575,니가 아빠냐?,0
100008,5585358,시간낭비. 차라리 포르노가 더 재밌을듯.,0
100009,9914590,"심리묘사가 중요한데, 이건 그냥 폭력만 나오네요.원작도 아쉬웠지만, 이건 그냥 쓰레기임",0


In [4]:
df.dropna(inplace = True) # inplace = True로 설정하면 데이터프레임에 즉시 적용됩니다.

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-1zq7epgx/kobert-tokenizer_a02eaa7df1324c5daa36e287bce82a35
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-1zq7epgx/kobert-tokenizer_a02eaa7df1324c5daa36e287bce82a35
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=fd1c6bfbabbfcfbcad70846f981faaedc6a2ce330cf6c524114ee5d24c20e1cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-_dgmhexd/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [6]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 128.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [7]:
from sklearn.model_selection import train_test_split



def split_data(df, train_size=150000, val_size=25000, test_size=25000, use_all_data=False):
    """
    데이터셋을 학습, 검증, 테스트 셋으로 분리합니다.

    사용법:컴퓨팅 자원에 따라 알아서 데이터 크기 조절
    recomended : M1 pro 기준 //  학습:10,000, 검증, 테스트: each set -> 2,000/2 = 1,000


    Parameters:
    - df: 전체 데이터셋 DataFrame
    - train_size: 학습 데이터셋 샘플 수 (use_all_data가 False일 때만 적용)
    - val_size: 검증 데이터셋 샘플 수
    - test_size: 테스트 데이터셋 샘플 수
    - use_all_data: True일 경우 전체 데이터를 사용하고, False일 경우 주어진 샘플 수만큼 사용합니다.

    Returns:
    - train_texts, train_labels: 학습 데이터셋 텍스트와 레이블 리스트
    - val_texts, val_labels: 검증 데이터셋 텍스트와 레이블 리스트
    - test_texts, test_labels: 테스트 데이터셋 텍스트와 레이블 리스트
    """
    if not use_all_data:
        sample_size = min(train_size + val_size + test_size, len(df))
        df_sample = df.sample(n=sample_size, random_state=42).reset_index(drop=True)
    else:
        df_sample = df  # 전체 데이터 사용

    # 학습, 검증, 테스트 셋 크기 계산
    train_ratio = train_size / (train_size + val_size + test_size)
    val_ratio = val_size / (val_size + test_size)

    # 학습 데이터 분리
    train_df, temp_df = train_test_split(df_sample, train_size=train_ratio, random_state=42)

    # 검증 및 테스트 데이터 분리
    val_df, test_df = train_test_split(temp_df, train_size=val_ratio, random_state=42)

    # 텍스트와 레이블을 리스트 형태로 변환
    train_texts = train_df['document'].astype(str).tolist()
    train_labels = train_df['label'].tolist()

    val_texts = val_df['document'].astype(str).tolist()
    val_labels = val_df['label'].tolist()

    test_texts = test_df['document'].astype(str).tolist()
    test_labels = test_df['label'].tolist()

    # 데이터셋 크기 확인 출력
    print(f"Train set size: {len(train_texts)}")
    print(f"Validation set size: {len(val_texts)}")
    print(f"Test set size: {len(test_texts)}")

    return train_texts, train_labels, val_texts, val_labels, test_texts, test_labels

# 함수 사용
#train_texts, train_labels, val_texts, val_labels, test_texts, test_labels = split_data(df, train_size=170000, val_size=15000, test_size=15000, use_all_data=True)

#if you are working on M1 PRO,
train_texts, train_labels, val_texts, val_labels, test_texts, test_labels = split_data(df, train_size=10000, val_size=1000, test_size=1000, use_all_data=False)

Train set size: 10000
Validation set size: 1000
Test set size: 1000


In [8]:
# 현재 설치된 PyTorch CUDA 버전에 맞춰 torchvision 재설치
#!pip install torchvision==0.14.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html // 이건 impress 연구실  내 컴퓨터 환경 : cuda 18
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 // 자동 설치
#!pip install --upgrade transformers sentencepiece
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel,BertForSequenceClassification #BertForSequenceClassification을 통해 2진분류
import torch

# KoBERT 모델과 토크나이저 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
import torch
from torch.utils.data import DataLoader, Dataset

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
batch_size = 32 # 배치 사이즈는 직접 지정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset,batch_size=batch_size , shuffle=False)


In [11]:
# sigmoid 사용하기 때문에 레이블은 1개로 지정합니다.
#model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1',num_labels=1)
# 2진 분류시 : CrossEntropy 사용
#model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1',num_labels=2,ignore_mismatched_sizes=True)
#model = model.to(device)

'''model = BertForSequenceClassification.from_pretrained(
    'skt/kobert-base-v1',
    num_labels=2,  # 이진 분류
    ignore_mismatched_sizes=True
)
model = model.to(device)  # GPU로 이동
print(f"Model device: {next(model.parameters()).device}")  # 디바이스 확인
'''


Available device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model device: cuda:0


In [ ]:
from tqdm.auto import tqdm
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
from transformers import get_linear_schedule_with_warmup
import logging
import os

# 로그 파일 설정
logging.basicConfig(filename='training_log.txt', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Training Started")

# 디바이스 설정
device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
print("Available device is:", device)
logging.info(f"Available device: {device}")

# 학습 파라미터 설정
num_epochs = 10  # 원하는 에포크 수 설정
learning_rate = 5e-5
warmup_ratio = 0.1
total_steps = len(train_loader) * num_epochs
warmup_steps = int(total_steps * warmup_ratio)

# 옵티마이저 및 스케줄러 설정
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
criterion = CrossEntropyLoss()
model.to(device)

# 학습 및 검증 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_train_predictions = 0
    total_train_predictions = 0

    # 학습 배치 루프
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).float()  # BCEWithLogitsLoss는 float형 입력 필요

        optimizer.zero_grad()

        # 모델 출력 및 손실 계산
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits.squeeze(), labels)

        # 역전파 및 파라미터 업데이트
        loss.backward()
        optimizer.step()

        # 학습률 스케줄러 업데이트
        scheduler.step()

        # 손실 누적
        total_loss += loss.item()

        # 학습 배치별 예측 정확도 계산
        probs = torch.sigmoid(outputs.logits.squeeze())
        predicted_labels = (probs >= 0.5).long()
        correct_train_predictions += (predicted_labels == labels).sum().item()
        total_train_predictions += labels.size(0)

    # 학습 정확도 및 평균 손실 계산
    train_accuracy = correct_train_predictions / total_train_predictions
    average_loss = total_loss / len(train_loader)

    # 검증 모드 전환
    model.eval()
    correct_val_predictions = 0
    total_val_predictions = 0

    # 검증 배치 루프
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # 모델 출력 및 예측 계산
            outputs = model(input_ids, attention_mask=attention_mask)
            probs = torch.sigmoid(outputs.logits.squeeze())  # 시그모이드 후 확률 계산
            predicted_labels = (probs >= 0.5).long()  # 0.5 임계값 적용

            correct_val_predictions += (predicted_labels == labels).sum().item()
            total_val_predictions += labels.size(0)

    # 검증 정확도 계산
    val_accuracy = correct_val_predictions / total_val_predictions

    # 에포크 손실 및 정확도 출력
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f} - Train Accuracy: {train_accuracy:.4f} - Validation Accuracy: {val_accuracy:.4f}")
    logging.info(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f} - Train Accuracy: {train_accuracy:.4f} - Validation Accuracy: {val_accuracy:.4f}")

    # 검증 정확도가 90% 이상이거나 마지막 에포크인 경우 모델 저장
    if val_accuracy >= 0.90 or epoch == num_epochs - 1:
        # 모델 저장 경로 설정
        model_save_path = f"../exp/koBERT_sigmoid_{val_accuracy:.3f}.pth"

        # 디렉터리가 없으면 생성
        os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved at {model_save_path}")
        logging.info(f"Model saved at {model_save_path}")

        # 검증 정확도가 95% 이상이면 조기 종료
        if val_accuracy >= 0.95:
            print(f"Early stopping at Epoch {epoch+1} as Validation Accuracy reached {val_accuracy:.4f}")
            logging.info(f"Early stopping at Epoch {epoch+1} as Validation Accuracy reached {val_accuracy:.4f}")
            break

logging.info("Training Finished")

model.to(device)
model.eval()
correct_predictions = 0
total_predictions = 0

# 검증 배치 루프
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 모델 출력 및 예측 계산
        outputs = model(input_ids, attention_mask=attention_mask)
        probs = torch.sigmoid(outputs.logits.squeeze())  # 시그모이드 후 확률 계산
        predicted_labels = (probs >= 0.5).long()  # 0.5 임계값 적용

        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0)

# 에포크 손실 및 정확도 출력
accuracy = correct_predictions / total_predictions
average_loss = total_loss / len(train_loader)
print(f"Test Accuracy: {accuracy:.4f}")

Available device is: cuda


Training Epoch 1/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 1/10 - Average Loss: 20.7587 - Train Accuracy: 0.6778 - Validation Accuracy: 0.5830


Training Epoch 2/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 2/10 - Average Loss: 19.5865 - Train Accuracy: 0.7818 - Validation Accuracy: 0.7940


...

Epoch 8/10 - Average Loss: 17.0458 - Train Accuracy: 0.9775 - Validation Accuracy: 0.8600


Training Epoch 9/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 9/10 - Average Loss: 16.9502 - Train Accuracy: 0.9836 - Validation Accuracy: 0.8620


Training Epoch 10/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 10/10 - Average Loss: 16.8945 - Train Accuracy: 0.9872 - Validation Accuracy: 0.8620
Model saved at ../exp/koBERT_sigmoid_0.862.pth
Test Accuracy: 0.8710


In [15]:
import torch
from transformers import BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss
from tqdm.auto import tqdm

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Available device: {device}")

# 모델 로드 및 GPU로 이동
model = BertForSequenceClassification.from_pretrained(
    'skt/kobert-base-v1',
    num_labels=2,  # 이진 분류를 위한 num_labels 설정
    ignore_mismatched_sizes=True
)
model.to(device)
print(f"Model device: {next(model.parameters()).device}")

# 옵티마이저 및 손실 함수
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = CrossEntropyLoss()

# 학습 루프
for epoch in range(1):  # 에포크 수를 1로 제한하여 디버깅
    model.train()
    for batch in tqdm(train_loader):
        # 데이터 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).long()

        # 레이블 값 확인
        print(f"Unique labels: {torch.unique(labels)}")  # 예상 결과: tensor([0, 1])

        # 모델 출력 및 크기 확인
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        print(f"logits shape: {logits.shape}")  # 예상: [batch_size, 2]
        print(f"labels shape: {labels.shape}")  # 예상: [batch_size]

        # 손실 계산
        loss = criterion(logits, labels)
        loss.backward()

        # 옵티마이저 업데이트
        optimizer.step()


Available device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model device: cuda:0


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/625 [00:00<?, ?it/s]

Unique labels: tensor([0, 1], device='cuda:0')
logits shape: torch.Size([16, 2])
labels shape: torch.Size([16])
Unique labels: tensor([0, 1], device='cuda:0')
logits shape: torch.Size([16, 2])
labels shape: torch.Size([16])
Unique labels: tensor([0, 1], device='cuda:0')
logits shape: torch.Size([16, 2])
labels shape: torch.Size([16])
Unique labels: tensor([0, 1], device='cuda:0')
logits shape: torch.Size([16, 2])
labels shape: torch.Size([16])
Unique labels: tensor([0, 1], device='cuda:0')
logits shape: torch.Size([16, 2])
labels shape: torch.Size([16])
Unique labels: tensor([0, 1], device='cuda:0')
logits shape: torch.Size([16, 2])
labels shape: torch.Size([16])
Unique labels: tensor([0, 1], device='cuda:0')
logits shape: torch.Size([16, 2])
labels shape: torch.Size([16])
Unique labels: tensor([0, 1], device='cuda:0')
logits shape: torch.Size([16, 2])
labels shape: torch.Size([16])
Unique labels: tensor([0, 1], device='cuda:0')
logits shape: torch.Size([16, 2])
labels shape: torch.Siz

In [18]:
import torch
from transformers import BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss
from tqdm.auto import tqdm

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Available device: {device}")

# 모델 로드 및 GPU로 이동
model = BertForSequenceClassification.from_pretrained(
    'skt/kobert-base-v1',
    num_labels=2,  # 이진 분류
    ignore_mismatched_sizes=True
)
model = model.to(device)  # GPU로 이동
print(f"Model device: {next(model.parameters()).device}")

# 옵티마이저 및 손실 함수
learning_rate = 5e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = CrossEntropyLoss()

# 학습 파라미터 설정
num_epochs = 10  # 원하는 에포크 수 설정
batch_size = 16
warmup_ratio = 0.1
total_steps = len(train_loader) * num_epochs
warmup_steps = int(total_steps * warmup_ratio)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_train_predictions = 0
    total_train_predictions = 0

    print(f"Epoch {epoch+1}/{num_epochs}")

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        # 데이터 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).long()

        # 레이블 값 확인


        optimizer.zero_grad()

        # 모델 출력
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits


        # 손실 계산
        loss = criterion(logits, labels)
        loss.backward()

        # 옵티마이저 업데이트
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # 그레이디언트 클리핑
        optimizer.step()

        # 손실 및 정확도 계산
        total_loss += loss.item()
        predicted_labels = logits.argmax(dim=1)
        correct_train_predictions += (predicted_labels == labels).sum().item()
        total_train_predictions += labels.size(0)

    # 에포크 결과 출력
    train_accuracy = correct_train_predictions / total_train_predictions
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Loss: {average_loss:.4f}, Accuracy: {train_accuracy:.4f}")

    # 검증 루프
    model.eval()
    total_val_loss = 0
    correct_val_predictions = 0
    total_val_predictions = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).long()

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            total_val_loss += loss.item()

            predicted_labels = logits.argmax(dim=1)
            correct_val_predictions += (predicted_labels == labels).sum().item()
            total_val_predictions += labels.size(0)

    val_accuracy = correct_val_predictions / total_val_predictions
    average_val_loss = total_val_loss / len(val_loader)
    print(f"Validation - Loss: {average_val_loss:.4f}, Accuracy: {val_accuracy:.4f}")


Available device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model device: cuda:0
Epoch 1/10


Training Epoch 1/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 1 - Loss: 0.6542, Accuracy: 0.6163
Validation - Loss: 0.6472, Accuracy: 0.5020
Epoch 2/10


...

Epoch 8 - Loss: 0.6959, Accuracy: 0.4999
Validation - Loss: 0.6940, Accuracy: 0.5080
Epoch 9/10


Training Epoch 9/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 9 - Loss: 0.6966, Accuracy: 0.4976
Validation - Loss: 0.6930, Accuracy: 0.5080
Epoch 10/10


Training Epoch 10/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 10 - Loss: 0.6971, Accuracy: 0.4961
Validation - Loss: 0.6934, Accuracy: 0.4920


In [20]:
from tqdm.auto import tqdm
from transformers import BertForSequenceClassification, AdamW, get_scheduler
from torch.nn import CrossEntropyLoss
import torch
import os
import logging

# 로그 파일 설정
logging.basicConfig(filename='training_log.txt', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Training Started")

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Available device:", device)
logging.info(f"Available device: {device}")

# 모델 로드 및 GPU로 이동
model = BertForSequenceClassification.from_pretrained(
    'skt/kobert-base-v1',
    num_labels=2,
    ignore_mismatched_sizes=True
)
model = model.to(device)

# 학습 파라미터 설정
num_epochs = 10
learning_rate = 1e-5  # 학습률 조정
weight_decay = 0.01  # L2 정규화 강도

# 옵티마이저 및 스케줄러 설정
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
num_training_steps = len(train_loader) * num_epochs
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

criterion = CrossEntropyLoss()  # 손실 함수 변경

# 학습 및 검증 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_train_predictions = 0
    total_train_samples = 0

    print(f"Epoch {epoch+1}/{num_epochs}")
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).long()  # 레이블은 정수형으로 변환

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # 크기: [batch_size, num_classes]

        # 손실 계산
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        # 손실 누적
        total_loss += loss.item()

        # 정확도 계산
        predictions = logits.argmax(dim=1)  # 가장 높은 확률의 클래스를 선택
        correct_train_predictions += (predictions == labels).sum().item()
        total_train_samples += labels.size(0)

    train_accuracy = correct_train_predictions / total_train_samples
    avg_loss = total_loss / len(train_loader)

    print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.4f}")
    logging.info(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.4f}")

    # 검증 루프
    model.eval()
    val_loss = 0
    correct_val_predictions = 0
    total_val_samples = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).long()

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            val_loss += loss.item()

            predictions = logits.argmax(dim=1)
            correct_val_predictions += (predictions == labels).sum().item()
            total_val_samples += labels.size(0)

    val_accuracy = correct_val_predictions / total_val_samples
    val_avg_loss = val_loss / len(val_loader)

    print(f"Validation - Loss: {val_avg_loss:.4f}, Accuracy: {val_accuracy:.4f}")
    logging.info(f"Validation - Loss: {val_avg_loss:.4f}, Accuracy: {val_accuracy:.4f}")

    # 모델 저장
    if epoch == num_epochs - 1 or val_accuracy >= 0.9:
        save_path = f"../exp/kobert_epoch{epoch+1}_valacc{val_accuracy:.3f}.pth"
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        torch.save(model.state_dict(), save_path)
        print(f"Model saved at {save_path}")
        logging.info(f"Model saved at {save_path}")

        # 조기 종료
        if val_accuracy >= 0.95:
            print(f"Early stopping at Epoch {epoch+1}")
            logging.info(f"Early stopping at Epoch {epoch+1}")
            break

logging.info("Training Finished")

# 테스트 루프
print("\nEvaluating on Test Dataset...")
model.eval()
test_loss = 0
correct_test_predictions = 0
total_test_samples = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).long()

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)
        test_loss += loss.item()

        predictions = logits.argmax(dim=1)
        correct_test_predictions += (predictions == labels).sum().item()
        total_test_samples += labels.size(0)

test_accuracy = correct_test_predictions / total_test_samples
test_avg_loss = test_loss / len(test_loader)

print(f"Test - Loss: {test_avg_loss:.4f}, Accuracy: {test_accuracy:.4f}")
logging.info(f"Test - Loss: {test_avg_loss:.4f}, Accuracy: {test_accuracy:.4f}")


Available device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10


Training Epoch 1/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 1 - Loss: 0.4943, Accuracy: 0.7556
Validation - Loss: 0.3571, Accuracy: 0.8460
Epoch 2/10


...

Epoch 8 - Loss: 0.0547, Accuracy: 0.9840
Validation - Loss: 0.4948, Accuracy: 0.8660
Epoch 9/10


Training Epoch 9/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 9 - Loss: 0.0424, Accuracy: 0.9877
Validation - Loss: 0.5444, Accuracy: 0.8750
Epoch 10/10


Training Epoch 10/10:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 10 - Loss: 0.0362, Accuracy: 0.9904
Validation - Loss: 0.5467, Accuracy: 0.8760
Model saved at ../exp/kobert_epoch10_valacc0.876.pth

Evaluating on Test Dataset...
Test - Loss: 0.5322, Accuracy: 0.8620
